In [1]:
import function.functions as functions
# import function.functions_nocuda as functions # use this if you can't use cuda and the GPU

from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cupy as cp

In [2]:
# Set device and parameters in this part

############################# Set parameters ##############################
device = 'transmon' # change here to change the device: charge_qubit, flopping_spin, flopping_charge, transmon, fluxonium

data = np.load('data/params/'+device+'.npz', allow_pickle=True)
H_sys, drive_op, wq, g, kappa, num_A, dim = data['H_sys'], data['drive_op'], data['wq'], data['g'], data['kappa'], data['num_A'], data['dim']
#############################################################################

########################### Parameters of the paper ###########################
if device == 'charge_qubit':
    num_w = 151

    N_rep = 60 # this means that we will have 2*N_rep+1 replicas

    n_states = 2

elif device == 'flopping_spin':
    num_w = 101
    
    N_rep = 15 # this means that we will have 2*N_rep+1 replicas

    n_states = 4

elif device == 'transmon':
    num_w = 101

    N_rep =  15 # this means that we will have 2*N_rep+1 replicas
    
    n_states = 20

elif device == 'fluxonium':
    num_w = 101

    N_rep = 15 # this means that we will have 2*N_rep+1 replicas

    n_states = 20

else:
    print('select a valid device')
    sys.exit()

final_t = 0.5

tlist = np.linspace(0,final_t/kappa,num=200)

ground, excited = 0, 1

H_sys = Qobj(H_sys[0:n_states,0:n_states])
drive_op = Qobj(drive_op[0:n_states,0:n_states])

compensation = False # set compensation True/False
###########################################################################

########################## Import data ####################################
fname_import = 'data/'+device+'/g_parallel/N_rep='+str(N_rep)+'_n_states='+str(n_states)\
        +'_dim='+str(dim)+'_num_w='+str(num_w)

def import_npz(npz_file):
    Data = np.load(npz_file, allow_pickle=True)
    for varName in Data:
        globals()[varName] = Data[varName]

import_npz(fname_import+'.npz')
###############################################################################

########################### Custom parameters #################################
# test other parameters wlist, Alist, compensation

save_file = True # test mode or save data to generate the data of the figure of the paper
################################################################################

In [3]:
############################## Search for optimal dispersive readout ###############################
#N_rep_optimal = 5
#chi_disp, index_wr_low, index_wr_high = functions.search_optimal_dispersive(N_rep_optimal,0.01*wq,H_sys,g,drive_op,wq,wlist,n_states,num_A,ground,excited,kappa)

#print(r'$\chi_{\text{disp}}/\kappa/2$='+str(chi_disp[index_wr_low]/(kappa/2))+', '+str(chi_disp[index_wr_high]/(kappa/2))\
#      +r' obtained at $\omega_r/\omega_q$='+str(wlist[index_wr_low]/wq)+' ,'+str(wlist[index_wr_high]/wq))

In [4]:
if save_file:
    fname = 'data/'+device+'/SNR_params_and_analytics_N_rep='+str(N_rep)+'_n_states='+str(n_states)\
        +'_dim='+str(dim)+'_num_w='+str(num_w)+'_final_t='+str(final_t)+'_compensation='+str(compensation)

    with open(fname+'.txt', "w") as data:
        message = '(dispersive) derivatives of the spectrum computed in A_q/w_q='+str(A_list[2]/wq)
        data.write(message+"\n")

chi1_disp = g**2*dd2_real[:,2,excited]
chi0_disp = g**2*dd2_real[:,2,ground]

chi_disp = chi1_disp-chi0_disp

if device == 'fluxonium':
    index_wr_low = np.argwhere(wlist > 0*wq)[0][0] \
        + (np.abs(np.abs(chi_disp[np.argwhere(wlist > 0)[0][0]:np.argwhere(wlist >= 45*wq)[0][0]])-kappa/2)).argmin() # find optimal dispersive readout low freq.

    index_wr_high = np.argwhere(wlist >= 45*wq)[0][0] \
        + (np.abs(np.abs(chi_disp[np.argwhere(wlist >= 45*wq)[0][0]:-1])-kappa/2)).argmin() # find optimal dispersive readout high freq.

else:
    index_wr_low = np.argwhere(wlist > 0.3*wq)[0][0] \
        + (np.abs(np.abs(chi_disp[np.argwhere(wlist > 0.3*wq)[0][0]:np.argwhere(wlist >= wq)[0][0]])-kappa/2)).argmin() # find optimal dispersive readout low freq.

    index_wr_high = np.argwhere(wlist >= wq)[0][0] \
        + (np.abs(np.abs(chi_disp[np.argwhere(wlist >= wq)[0][0]:-1])-kappa/2)).argmin() # find optimal dispersive readout high freq.

message = r'$\chi_{\text{disp}}/\kappa/2$='+str(chi_disp[index_wr_low]/(kappa/2))+', '+str(chi_disp[index_wr_high]/(kappa/2))\
      +r' obtained at $\omega_r/\omega_q$='+str(wlist[index_wr_low]/wq)+' ,'+str(wlist[index_wr_high]/wq)

print(message)

if fname != None:
    with open(fname+'.txt', "a") as data:
        data.write(message+"\n")


$\chi_{\text{disp}}/\kappa/2$=-0.9283772792814883, -0.9437342900976661 obtained at $\omega_r/\omega_q$=0.74 ,1.3


In [5]:
res_an = np.zeros((len(Alist),len(wlist)), dtype=object)

A_r_array = np.zeros((len(Alist),len(wlist)))

A_d_array = np.zeros((len(Alist),len(wlist)))
w_d_array= np.zeros((len(Alist),len(wlist)))

w_r_disp_array= np.zeros((len(wlist)))
w_d_disp_array= np.zeros((len(wlist)))

temp_chi = np.zeros((len(wlist)))

temp_chi[wlist < wq] = chi_disp[index_wr_low]
temp_chi[wlist >= wq] = chi_disp[index_wr_high]

if(compensation):
    w_d_disp_array = w_r_disp_array+chi_sum
else:
    w_d_disp_array = w_r_disp_array

for i, A_q in enumerate(Alist):    
    index_A = np.abs(A_list[0:(num_A-2)]-A_q).argmin()

    print('derivatives of the spectrum computed in A_q/w_q='+str(A_list[index_A]/wq))

    g0 = g*(dd_real[:,index_A,0])
    g1 = g*(dd_real[:,index_A,1])
    
    chi0 = g**2*(dd2_real[:,index_A,0]+1/A_q*dd_real[:,index_A,0])        
    chi1 = g**2*(dd2_real[:,index_A,1]+1/A_q*dd_real[:,index_A,1])

    g_parallel = 1/2*(g1-g0)
    chi = 1/2*(chi1-chi0)

    g_sum = 1/2*(g1+g0)
    chi_sum = 1/2*(chi1+chi0)
    
    gamma = 0 # too short time for Purcell to matters

    w_r_disp_array[wlist < wq] = wlist[index_wr_low]
    w_r_disp_array[wlist >= wq] = wlist[index_wr_high]

    if(compensation):
        w_d_array[i,:] = wlist+chi_sum # shifted resonator frequency
        A_d_array[i,:] = -2*g_sum # compensation tone
    else:
        w_d_array[i,:] = wlist

    A_r_array[i,:] = np.abs(g_parallel*kappa/temp_chi*(temp_chi**2+kappa**2/4)/(chi**2+kappa**2/4))

    for j, w_r in enumerate(wlist):

        res_a_an = [[],[]]

        for idx, qubit_state in enumerate([ground,excited]):

            z0 = z0_list[j,i,qubit_state]
            
            analytical_result = functions.analytical_time_dynamics(z0,w_r,w_d_array[i,j],A_d_array[i,j],0,g_parallel[j],g_sum[j],chi[j],chi_sum[j],kappa,gamma,tlist)
            res_a_an[idx].append(analytical_result)

        res_an[i,j] = functions.generate_SNR_list(res_a_an,kappa,tlist)[-1]

derivatives of the spectrum computed in A_q/w_q=0.02040579710144928
derivatives of the spectrum computed in A_q/w_q=0.039536231884057985
derivatives of the spectrum computed in A_q/w_q=0.08034782608695654


In [6]:
if save_file:
    np.savez(fname, Alist=Alist, wlist=wlist, res_an=res_an, A_r_array=A_r_array, A_d_array=A_d_array,\
            w_d_array=w_d_array, w_r_disp_array=w_r_disp_array, w_d_disp_array=w_d_disp_array, \
                index_wr_low=index_wr_low, index_wr_high=index_wr_high, chi_disp=chi_disp)